# Entraînement du Neural Network pour Smart Chess sur Google Colab

Ce notebook permet d'entraîner le réseau de neurones pour l'évaluation d'échecs en utilisant les ressources GPU de Google Colab.

**Chemin du projet sur Drive:** `MyDrive/smart_chess_drive/smart-chess`

## Instructions
1. Aller dans **Runtime > Change runtime type > GPU** (T4 ou mieux)
2. Exécuter les cellules dans l'ordre
3. Les modèles seront sauvegardés automatiquement sur votre Drive

## 1. Vérification GPU

In [2]:
# Vérifier la disponibilité du GPU
!nvidia-smi

Sun Nov 16 16:56:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   66C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2. Montage Google Drive

In [3]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 3. Configuration du chemin du projet

In [4]:
# Définir le chemin vers le projet sur votre Drive
import os
import sys

PROJECT_PATH = '/content/drive/MyDrive/smart_chess_drive/smart-chess'
os.chdir(PROJECT_PATH)
sys.path.insert(0, PROJECT_PATH)

print(f"Répertoire de travail: {os.getcwd()}")
print(f"\nContenu du répertoire:")
for item in sorted(os.listdir('.')):
    print(f"  - {item}")

Répertoire de travail: /content/drive/MyDrive/smart_chess_drive/smart-chess

Contenu du répertoire:
  - .git
  - .gitignore
  - README.md
  - ai
  - docs
  - prototypes


## 4. Installation des dépendances

In [5]:
# Installer les packages nécessaires
!pip install -q torch torchvision torchaudio
!pip install -q numpy matplotlib tqdm

print("✓ Installation terminée")

✓ Installation terminée


## 5. Vérification de l'environnement PyTorch

In [6]:
import torch
import numpy as np

print("=" * 60)
print("CONFIGURATION SYSTÈME")
print("=" * 60)
print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"\nCUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Nom du GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    print(f"Mémoire GPU totale: {props.total_memory / 1e9:.2f} GB")
    print(f"Compute Capability: {props.major}.{props.minor}")
else:
    print("⚠️ ATTENTION: GPU non disponible, l'entraînement sera très lent!")
    print("   Allez dans Runtime > Change runtime type > GPU")

print("=" * 60)

CONFIGURATION SYSTÈME
PyTorch version: 2.8.0+cu126
NumPy version: 2.0.2

CUDA disponible: True
CUDA version: 12.6
Nom du GPU: Tesla T4
Mémoire GPU totale: 15.83 GB
Compute Capability: 7.5


## 6. Import des modules du projet

In [7]:
# Importer les modules nécessaires depuis le projet (robuste à l'emplacement du repo sur Drive)
import os
import sys
import importlib

# Assurez-vous que PROJECT_PATH est défini et ajoutez également le dossier `ai` au PYTHONPATH
PROJECT_PATH = '/content/drive/MyDrive/smart_chess_drive/smart-chess'
AI_SUBDIR = os.path.join(PROJECT_PATH, 'ai')

# Vérifier les chemins alternatifs (si l'utilisateur a copié le repo dans /content)
ALT_PATH = '/content/smart-chess'

# Choisir un chemin existant
if not os.path.isdir(PROJECT_PATH) and os.path.isdir(ALT_PATH):
    PROJECT_PATH = ALT_PATH

if not os.path.isdir(PROJECT_PATH):
    raise FileNotFoundError(f"Répertoire projet introuvable: {PROJECT_PATH}. Montez Drive et vérifiez le chemin.")

# Ajouter au sys.path si nécessaire
if PROJECT_PATH not in sys.path:
    sys.path.insert(0, PROJECT_PATH)
if AI_SUBDIR not in sys.path and os.path.isdir(AI_SUBDIR):
    sys.path.insert(0, AI_SUBDIR)

# Se placer dans le répertoire projet
os.chdir(PROJECT_PATH)

print('Répertoire de travail:', os.getcwd())
print('\nQuelques fichiers à la racine du projet:')
print(sorted(os.listdir(PROJECT_PATH))[:50])
print('\nContenu du dossier ai/:')
print(sorted(os.listdir(AI_SUBDIR))[:100])

# Diagnostic d'import direct pour le module Chess
try:
    import Chess
    print('\n✅ Import direct `Chess` OK (module trouvé via sys.path)')
except Exception as e:
    print('\n❌ Import direct `Chess` a échoué:', e)
    print('Vérifiez que `ai/Chess.py` existe et que le dossier ai/ est dans sys.path')

# Maintenant importer le module d'entraînement (trainer) - UPDATED to torch_train
try:
    import ai.NN.torch_train as trainer
    import ai.NN.torch_nn_evaluator as torch_eval
    from ai.Chess_v2 import Chess
    print('\n✓ Modules importés avec succès!')
except Exception as e:
    print('\n❌ Erreur d\'import lors de l\'import du trainer:', e)
    raise


Répertoire de travail: /content/drive/MyDrive/smart_chess_drive/smart-chess

Quelques fichiers à la racine du projet:
['.git', '.gitignore', 'README.md', 'ai', 'docs', 'prototypes']

Contenu du dossier ai/:
['AI_reduction', 'Chess.py', 'ChessInteractif - v7.py', 'ChessInteractifv10.py', 'ChessInteractifv2.py', 'Chess_v2.py', 'NN', 'Null_move_AI', 'Old_AI', 'Player.py', 'Profile', 'Tests.py', '__init__.py', '__pycache__', 'alphabeta.py', 'alphabeta_engine.py', 'alphabeta_engine_v2.py', 'analyze_reduction_overhead.py', 'base_engine.py', 'check_dataset_stats.py', 'check_gpu.py', 'check_performance.py', 'checkpoints', 'chess_model_checkpoint.pt', 'debug_conversion.py', 'engine.py', 'engine_match.py', 'evaluator.py', 'example_move_reduction.py', 'fast_evaluator.py', 'gaviota.py', 'journal-experiments.md', 'optimized_chess.py', 'pgn.py', 'polyglot.py', 'profile_report_1760344602.txt', 'py.typed', 'svg.py', 'syzygy.py', 'test_depth_6_performance.py', 'test_depth_6_quick.py', 'test_depth_effec

## 7. Configuration de l'entraînement

In [8]:
# Paramètres d'entraînement (NNUE architecture)
CONFIG = {
    # Génération de données
    'num_games': 10000,          # Nombre de parties à générer pour l'entraînement

    # Hyperparamètres NNUE
    'batch_size': 256,           # Taille du batch (augmenter si GPU puissant)
    'epochs': 50,                # Nombre d'époques d'entraînement
    'learning_rate': 0.001,      # Taux d'apprentissage

    # Architecture NNUE (768 → 4096 → 256 → 32 → 1)
    'hidden1': 4096,
    'hidden2': 256,
    'hidden3': 32,
    'dropout': 0.0,              # NNUE ne use pas de dropout

    # Configuration système
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'num_workers': 2,            # Workers pour le DataLoader

    # Sauvegarde
    'checkpoint_path': 'ai/chess_model_checkpoint.pt',
    'save_interval': 5,          # Sauvegarder tous les N époques
}

print("=" * 60)
print("CONFIGURATION DE L'ENTRAÎNEMENT (NNUE)")
print("=" * 60)
for key, value in CONFIG.items():
    print(f"{key:20s}: {value}")
print("=" * 60)

if CONFIG['device'] == 'cpu':
    print("\n⚠️ ATTENTION: Entraînement sur CPU détecté!")
    print("   Réduisez num_games et epochs pour un test rapide.")


CONFIGURATION DE L'ENTRAÎNEMENT (NNUE)
num_games           : 10000
batch_size          : 256
epochs              : 50
learning_rate       : 0.001
hidden1             : 4096
hidden2             : 256
hidden3             : 32
dropout             : 0.0
device              : cuda
num_workers         : 2
checkpoint_path     : ai/chess_model_checkpoint.pt
save_interval       : 5


## 8. Génération des données d'entraînement

Cette étape génère des parties d'échecs aléatoires et calcule les évaluations de position.
**Attention:** Cela peut prendre 15-30 minutes selon le nombre de parties.

In [9]:
# Localiser le dataset sur Google Drive et préparer le dossier de checkpoints
import os
from glob import glob

# Chemin attendu du dossier contenant le dataset (donné par l'user)
# Updated based on user's feedback that the file is directly in smart_chess_drive
DATASET_DIR = '/content/drive/MyDrive/smart_chess_drive/'

# Chercher un fichier .csv dans DATASET_DIR
DATASET_CSV = None
if os.path.exists(DATASET_DIR):
    csvs = glob(os.path.join(DATASET_DIR, '*.csv'))
    if len(csvs) > 0:
        # Assuming there's only one relevant CSV in that dir, pick the first one
        DATASET_CSV = csvs[0]
        print(f'✅ Dataset CSV trouvé: {DATASET_CSV}')
    else:
        print(f'❌ Aucun fichier .csv trouvé dans {DATASET_DIR}. Placez votre fichier chessData.csv dans ce dossier.')
else:
    print(f'❌ Dossier dataset introuvable: {DATASET_DIR}. Vérifiez le chemin sur votre Drive.')

# Créer un dossier de checkpoints dans le repo sur Drive (persistant)
CKPT_DIR = '/content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints'
os.makedirs(CKPT_DIR, exist_ok=True)
print('Dossier de checkpoints (créé si manquant):', CKPT_DIR)

# Exposer variables utiles
print('\nVariables exposées:')
print(' DATASET_CSV =', DATASET_CSV)
print(' CKPT_DIR =', CKPT_DIR)

✅ Dataset CSV trouvé: /content/drive/MyDrive/smart_chess_drive/chessData.csv
Dossier de checkpoints (créé si manquant): /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints

Variables exposées:
 DATASET_CSV = /content/drive/MyDrive/smart_chess_drive/chessData.csv
 CKPT_DIR = /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints


In [10]:
from tqdm import tqdm
import time
from sklearn.model_selection import train_test_split

print("Chargement du dataset (depuis chessData)...")

# Préférer la variable DATASET_CSV (définie après le montage Drive) sinon utiliser la valeur par défaut du module trainer
dataset_path = globals().get('DATASET_CSV')

if dataset_path is None:
    raise FileNotFoundError('Aucun chemin de dataset défini. Montez Drive et placez le fichier CSV dans MyDrive/smart_chess_drive/chessData')

start_time = time.time()

# Utiliser la fonction de chargement du script d'entraînement pour assurer le même prétraitement
fens, evaluations = trainer.load_data(dataset_path)

# Variables attendues plus bas dans le notebook (split train/validation: 99% train, 1% val)
X_train, X_val, y_train, y_val = train_test_split(fens, evaluations, test_size=0.01, random_state=42)

elapsed_time = time.time() - start_time

print("\n" + "=" * 60)
print("DONNÉES CHARGÉES")
print("=" * 60)
print(f"Dataset train: {len(X_train):,} positions")
print(f"Dataset validation: {len(X_val):,} positions")
print(f"Total: {len(fens):,} positions")
print(f"Temps écoulé: {elapsed_time:.1f}s ({elapsed_time/60:.1f} min)")
print("=" * 60)



Chargement du dataset (depuis chessData)...
📂 Chargement du dataset depuis /content/drive/MyDrive/smart_chess_drive/chessData.csv...
🧹 Nettoyage : 190154 lignes corrompues supprimées.
✅ 12,767,881 positions valides chargées.

DONNÉES CHARGÉES
Dataset train: 12,640,202 positions
Dataset validation: 127,679 positions
Total: 12,767,881 positions
Temps écoulé: 19.7s (0.3 min)


In [11]:
import inspect
import ai.NN.torch_train as trainer

try:
    # Get the source code of the load_data function
    source_code = inspect.getsource(trainer.load_data)
    print("Source code of trainer.load_data:")
    print("=" * 60)
    print(source_code)
    print("=" * 60)
except TypeError:
    print("Could not get source code for trainer.load_data. It might not be a function defined in the file.")
except FileNotFoundError:
    print("Could not find the torch_train.py file.")
except Exception as e:
    print(f"An error occurred while trying to get source code: {e}")


Source code of trainer.load_data:
def load_data(filepath: str):
    """Charge le dataset FEN,Evaluation et le nettoie."""
    print(f"📂 Chargement du dataset depuis {filepath}...")
    
    df = pd.read_csv(
        filepath, 
        names=['FEN', 'Evaluation'], 
        skiprows=1,
        comment='#'
    )
    
    initial_count = len(df)
    df.dropna(inplace=True)
    cleaned_count = len(df)
    
    if initial_count > cleaned_count:
        print(f"🧹 Nettoyage : {initial_count - cleaned_count} lignes corrompues supprimées.")
    
    fens = df['FEN'].values
    EVAL_SCALE_FACTOR = 1000.0
    evaluations = (df['Evaluation'].astype(int).values) / EVAL_SCALE_FACTOR
    
    print(f"✅ {len(fens):,} positions valides chargées.")
    return fens, evaluations



In [12]:
import os

file_path = os.path.join(PROJECT_PATH, 'ai/NN/torch_train.py')

# Read the content of the file
with open(file_path, 'r') as f:
    content = f.read()

# Assuming the load_data function signature is currently load_data(filepath: str):
# We need to verify it accepts a filepath parameter
if 'def load_data(filepath:' in content or 'def load_data(filepath)' in content:
    print(f"✅ La fonction load_data dans {file_path} accepte déjà un paramètre filepath.")
    print("Aucune modification nécessaire.")
else:
    print(f"⚠️ La fonction load_data pourrait nécessiter une modification.")
    print("Vérifiez manuellement si elle accepte un chemin de fichier en paramètre.")


✅ La fonction load_data dans /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/NN/torch_train.py accepte déjà un paramètre filepath.
Aucune modification nécessaire.


## 9. Création du dataset et du dataloader

In [13]:
from torch.utils.data import DataLoader
from ai.NN.torch_train import ChessDataset

# Créer le dataset d'entraînement
dataset = ChessDataset(X_train, y_train)

# Créer le dataloader d'entraînement
train_loader = DataLoader(
    dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    pin_memory=True if CONFIG['device'] == 'cuda' else False
)

print("=" * 60)
print("DATALOADER CONFIGURÉ")
print("=" * 60)
print(f"Taille du dataset d'entraînement: {len(dataset):,} échantillons")
print(f"Nombre de batches: {len(train_loader):,}")
print(f"Taille du batch: {CONFIG['batch_size']}")
print(f"Dernière batch: {len(dataset) % CONFIG['batch_size']} échantillons")
print(f"\nDataset de validation: {len(X_val):,} échantillons")
print("=" * 60)



DATALOADER CONFIGURÉ
Taille du dataset d'entraînement: 12,640,202 échantillons
Nombre de batches: 49,376
Taille du batch: 256
Dernière batch: 202 échantillons

Dataset de validation: 127,679 échantillons


## 10. Création du modèle

In [14]:
# Créer le modèle NNUE et le déplacer sur le device approprié
from ai.NN.torch_nn_evaluator import TorchNNEvaluator

model = TorchNNEvaluator(
    hidden1=CONFIG['hidden1'],
    hidden2=CONFIG['hidden2'],
    hidden3=CONFIG['hidden3'],
    dropout=CONFIG['dropout']
).to(CONFIG['device'])

# Afficher l'architecture
print("=" * 60)
print("ARCHITECTURE DU MODÈLE (NNUE-LIKE)")
print("=" * 60)
print(model)
print("=" * 60)

# Compter les paramètres
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nNombre total de paramètres: {total_params:,}")
print(f"Paramètres entraînables: {trainable_params:,}")
print(f"Device: {CONFIG['device']}")

# Estimer la taille mémoire du modèle
param_size_mb = total_params * 4 / (1024 ** 2)  # 4 bytes par float32
print(f"Taille estimée du modèle: {param_size_mb:.2f} MB")

# Afficher les dimensions des couches
print(f"\nArchitecture détaillée:")
print(f"  Input:  {model.l1.in_features}")
print(f"  Layer 1: {model.l1.out_features} (ReLU)")
print(f"  Layer 2: {model.l2.out_features} (ReLU)")
print(f"  Layer 3: {model.l3.out_features} (ReLU)")
print(f"  Output: {model.l4.out_features} (Linear)")


ARCHITECTURE DU MODÈLE (NNUE-LIKE)
TorchNNEvaluator(
  (l1): Linear(in_features=768, out_features=4096, bias=True)
  (l2): Linear(in_features=4096, out_features=256, bias=True)
  (l3): Linear(in_features=256, out_features=32, bias=True)
  (l4): Linear(in_features=32, out_features=1, bias=True)
  (act): ReLU()
)

Nombre total de paramètres: 4,206,913
Paramètres entraînables: 4,206,913
Device: cuda
Taille estimée du modèle: 16.05 MB

Architecture détaillée:
  Input:  768
  Layer 1: 4096 (ReLU)
  Layer 2: 256 (ReLU)
  Layer 3: 32 (ReLU)
  Output: 1 (Linear)


## 11. Entraînement du modèle

Cette étape lance l'entraînement complet. Les checkpoints sont sauvegardés automatiquement sur votre Drive.

In [ ]:
# Configurer et lancer le script d'entraînement `ai.NN.torch_train` en adaptant les chemins pour Colab/Drive
import os
import importlib

if DATASET_CSV is None:
    raise FileNotFoundError(f"Dataset non trouvé dans: {DATASET_DIR}")

# Importer le module d'entraînement (UPDATED: torch_train au lieu de train_torch)
import ai.NN.torch_train as trainer

# Reload the module to pick up recent changes
importlib.reload(trainer)

# Rediriger les chemins dataset et checkpoints vers Drive
trainer.DATASET_PATH = DATASET_CSV
trainer.CHECKPOINT_FILE = os.path.join(CKPT_DIR, os.path.basename(trainer.CHECKPOINT_FILE))
trainer.WEIGHTS_FILE = os.path.join(CKPT_DIR, os.path.basename(trainer.WEIGHTS_FILE))

# Harmonisation des paramètres avec CONFIG
try:
    trainer.BATCH_SIZE = CONFIG['batch_size']
    print(f'✅ Harmonisation: trainer.BATCH_SIZE = {trainer.BATCH_SIZE}')
except Exception as e:
    print('⚠️ Impossible de définir trainer.BATCH_SIZE:', e)

# Appliquer l'architecture NNUE
try:
    trainer.HIDDEN1 = CONFIG['hidden1']
    trainer.HIDDEN2 = CONFIG['hidden2']
    trainer.HIDDEN3 = CONFIG['hidden3']
    trainer.DROPOUT = CONFIG['dropout']
    print(f"✅ Architecture NNUE appliquée: {trainer.HIDDEN1} → {trainer.HIDDEN2} → {trainer.HIDDEN3}")
except Exception as e:
    print('⚠️ Impossible de définir l\'architecture NNUE:', e)

# Optionnellement ajuster MAX_SAMPLES
try:
    trainer.MAX_SAMPLES = 200_000
    print(f"✅ MAX_SAMPLES = {trainer.MAX_SAMPLES}")
except Exception as e:
    print('⚠️ Impossible de définir trainer.MAX_SAMPLES:', e)

# Optionnel: réduire pour test rapide (décommentez si besoin)
# trainer.EPOCHS = 2
# trainer.MAX_SAMPLES = 5000

print('\nConfiguration trainer:')
print(' DATASET_PATH=', trainer.DATASET_PATH)
print(' CHECKPOINT_FILE=', trainer.CHECKPOINT_FILE)
print(' WEIGHTS_FILE=', trainer.WEIGHTS_FILE)
print(' Architecture: 768 →', trainer.HIDDEN1, '→', trainer.HIDDEN2, '→', trainer.HIDDEN3, '→ 1')
print(' EPOCHS=', trainer.EPOCHS)
print(' MAX_SAMPLES=', trainer.MAX_SAMPLES)

# Lancer l'entraînement
trainer.main()


🖥️  Device: cuda
🚀 GPU: Tesla T4
💾 GPU Memory: 15.83 GB
✅ Harmonisation: trainer.BATCH_SIZE = 256
✅ Architecture NNUE appliquée: 4096 → 256 → 32
✅ MAX_SAMPLES = 200000

Configuration trainer:
 DATASET_PATH= /content/drive/MyDrive/smart_chess_drive/chessData.csv
 CHECKPOINT_FILE= /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints/chess_model_checkpoint.pt
 WEIGHTS_FILE= /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints/chess_nn_weights.npz
 Architecture: 768 → 4096 → 256 → 32 → 1
 EPOCHS= 10
 MAX_SAMPLES= 200000
📂 Chargement du dataset depuis /content/drive/MyDrive/smart_chess_drive/chessData.csv...
🧹 Nettoyage : 190154 lignes corrompues supprimées.
✅ 12,767,881 positions valides chargées.

📊 Dataset complet: 12,767,881 positions
📊 Train: 12,640,202 positions
📊 Validation: 127,679 positions
📥 Chargement du checkpoint PyTorch: /content/drive/MyDrive/smart_chess_drive/smart-chess/ai/checkpoints/chess_model_checkpoint.pt
✅ Checkpoint chargé (step 10), bes

Epoch 1/10:   0%|          | 0/782 [00:00<?, ?it/s]

[GRAD] epoch=1 batch=0 grad_norm=0.794061 max_abs_grad=0.554259 param_norm=2863.824657


Epoch 1/10:   1%|          | 8/782 [00:00<00:28, 26.78it/s, loss=0.4939]


[DEBUG batch 0] targets mean=-0.0036 std=0.6496; preds mean=0.0153 std=0.4761; RMSE=0.4593; corr=0.7082


...

Epoch 10/10:  14%|█▍        | 111/782 [00:01<00:10, 62.71it/s, loss=0.5265]

[GRAD] epoch=10 batch=100 grad_norm=1.666374 max_abs_grad=1.355342 param_norm=2864.119222


Epoch 10/10:  27%|██▋       | 209/782 [00:03<00:10, 52.38it/s, loss=0.5187]

[GRAD] epoch=10 batch=200 grad_norm=1.241821 max_abs_grad=1.018399 param_norm=2864.122207


Epoch 10/10:  29%|██▊       | 223/782 [00:03<00:09, 58.72it/s, loss=0.5186]

## 14. Test du modèle sur des positions aléatoires

In [ ]:
# @title
# Passer le modèle en mode évaluation
model.eval()

# Tester sur quelques positions aléatoires DU DATASET DE VALIDATION
num_tests = 10
test_indices = np.random.choice(len(X_val), num_tests, replace=False)

print("=" * 60)
print(f"TEST SUR {num_tests} POSITIONS ALÉATOIRES (VALIDATION SET)")
print("=" * 60)

errors = []

with torch.no_grad():
    for i, idx in enumerate(test_indices, 1):
        # Créer un dataset temporaire pour encoder la position
        temp_dataset = ChessDataset([X_val[idx]], [y_val[idx]])
        x, _ = temp_dataset[0]
        x = x.unsqueeze(0).to(CONFIG['device'])

        y_true = y_val[idx]
        y_pred = model(x).cpu().numpy()[0, 0]
        error = abs(y_true - y_pred)
        errors.append(error)

        print(f"\nPosition {i}:")
        print(f"  Évaluation réelle:  {y_true:+8.4f}")
        print(f"  Prédiction modèle:  {y_pred:+8.4f}")
        print(f"  Erreur absolue:     {error:8.4f}")

        # Indicateur visuel de la qualité
        if error < 0.1:
            print(f"  Qualité: ✅ Excellente")
        elif error < 0.3:
            print(f"  Qualité: ✓ Bonne")
        elif error < 0.5:
            print(f"  Qualité: ⚠ Moyenne")
        else:
            print(f"  Qualité: ❌ Faible")

print("\n" + "=" * 60)
print("STATISTIQUES DES TESTS")
print("=" * 60)
print(f"Erreur moyenne: {np.mean(errors):.4f}")
print(f"Erreur médiane: {np.median(errors):.4f}")
print(f"Erreur min:     {np.min(errors):.4f}")
print(f"Erreur max:     {np.max(errors):.4f}")
print(f"Écart-type:     {np.std(errors):.4f}")
print("=" * 60)
